In [1]:
from fjss import FJSS2, FJSS3
import numpy as np
import time

from itertools import product
from collections import OrderedDict

from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

from utils import get_m_value, parse_data
import pandas as pd

In [2]:
infinity = 1.0e6

In [3]:
def single_run(n_opt_selected=10, method="CP"):
    # load the data
    (
        n_opt,
        n_mach,
        operation_data,
        machine_data,
        para_lmin,
        para_lmax,
        para_p,
        para_h,
        para_w,
        para_delta,
        para_a,
        para_mach_capacity,
        big_m,
    ) = load_data()

    n_opt = n_opt_selected

    para_p_horizon = np.copy(para_p[:n_opt_selected, :])
    para_p_horizon[para_p_horizon == np.inf] = 0

    # para_h = np.empty((n_opt, n_mach), dtype=object)
    para_h_horizon = np.copy(para_h[:n_opt_selected, :])
    para_h_horizon[para_h_horizon == np.inf] = 0

    para_lmax_horizon = np.copy(para_lmax[:n_opt_selected, :n_opt_selected])
    para_lmax_horizon[para_lmax_horizon == np.inf] = 0

    horizon = (
        np.sum(para_p_horizon, axis=1)
        + np.sum(para_h_horizon, axis=1)
        + np.sum(para_lmax_horizon, axis=1)
    )
    horizon = int(np.sum(horizon)) + 1

    para_lmin[para_lmin == -np.inf] = -infinity
    para_lmin[para_lmin == np.inf] = infinity
    para_lmax[para_lmax == np.inf] = infinity
    para_lmax[para_lmax == -np.inf] = -infinity

    para_p[para_p == np.inf] = infinity
    para_p[para_p == -np.inf] = -infinity
    para_w[para_w == np.inf] = infinity
    para_w[para_w == -np.inf] = -infinity

    para_a[para_a == np.inf] = infinity
    para_a[para_a == -np.inf] = -infinity

    # =====================================================
    # convert all the numpy arrays with integers data type
    para_lmin = para_lmin.astype(int)
    para_lmax = para_lmax.astype(int)
    para_p = para_p.astype(int)
    para_w = para_w.astype(int)
    para_a = para_a.astype(int)
    para_delta = para_delta.astype(int)
    para_mach_capacity = para_mach_capacity.astype(int)

    para_lmin = para_lmin[:n_opt_selected, :n_opt_selected]
    para_lmax = para_lmax[:n_opt_selected, :n_opt_selected]

    para_p = para_p[:n_opt_selected, :]
    para_h = para_h[:n_opt_selected, :]
    para_w = para_w[:n_opt_selected, :]

    para_a = para_a[:n_opt_selected, :n_opt_selected, :]

    # machines
    machines = [str(i) for i in range(6)]
    # operations
    operations = [str(i) for i in range(n_opt_selected)]

    operations = operations[:n_opt_selected]
    machines = machines[:n_mach]

    # =====================================================
    if method == "CP":
        # time the running time for the CP model
        start_time = time.time()
        fjss2 = FJSS2(
            operations=operations,
            machines=machines,
            para_p=para_p,
            para_a=para_a,
            para_w=para_w,
            para_h=para_h,
            para_delta=para_delta,
            para_mach_capacity=para_mach_capacity,
            para_lmin=para_lmin,
            para_lmax=para_lmax,
            precedence=None,
            model_string=None,
            inf_cp=1.0e6,
            num_workers=4,
            verbose=True,
        )
        fjss2.build_model_ortools()
        fjss2.solve_ortools()
        # get the running time in seconds
        running_time = time.time() - start_time

        new_row = {
            "method": method,
            "n_opt": n_opt_selected,
            "n_mach": n_mach,
            "running_time_seconds": running_time,
            "num_constraints": 0,
            "makespan": fjss2.var_c_max,
        }
    elif method == "MILP":
        para_a = np.einsum("mij->ijm", para_a)
        # time the running time for the MILP model
        start_time = time.time()
        fjss3 = FJSS3(
            operations=operations,
            machines=machines,
            para_p=para_p,
            para_a=para_a,
            para_w=para_w,
            para_h=para_h,
            para_delta=para_delta,
            para_mach_capacity=para_mach_capacity,
            para_lmin=para_lmin,
            para_lmax=para_lmax,
            precedence=None,
            model_string=None,
            inf_milp=1.0e7,
            num_workers=4,
            verbose=True,
        )
        fjss3.solve_gurobi()
        running_time = time.time() - start_time
        new_row = {
            "method": method,
            "n_opt": n_opt_selected,
            "n_mach": n_mach,
            "running_time_seconds": running_time,
            "num_constraints": fjss3.solver.NumConstraints(),
            "makespan": fjss3.var_c_max,
        }
    else:
        raise ValueError("Invalid method!")

    return new_row


def load_data():
    n_opt, n_mach, operation_data, machine_data = parse_data(
        "fjss_cp_data/gfjsp_10_5_1.txt"
    )
    operation_data["0"]["h"] = 0
    operation_data["1"]["h"] = 0
    operation_data["2"]["h"] = 0
    operation_data["3"]["h"] = 20

    # define the parameters

    # minimum lag between the starting time of operation i and the ending time of operation j
    para_lmin = np.full((n_opt, n_opt), dtype=object, fill_value=-np.inf)
    # maximum lag between the starting time of operation i and the ending time of operation j
    para_lmax = np.full((n_opt, n_opt), dtype=object, fill_value=np.inf)

    # processing time of operation i in machine m
    # para_p = np.full((n_opt, n_mach), dtype=object, fill_value=np.inf)
    para_p = np.full((n_mach, n_opt), dtype=object, fill_value=np.inf)

    # the shape of h in the original file is (n_machine) while the shape of para_h in the
    # paper is (n_opt, n_mach)
    # maximum holding time of operation i in machine m
    para_h = np.empty((n_opt, n_mach), dtype=object)
    # para_h = np.empty(n_mach, dtype=object)

    # mapping of operation i to machine m
    # 20 for the furnaces, 0 for Cutting, Pressing, and Forging
    # 0: Cutting; 1: Pressing; 2: Forging; 3: Furnace
    holding_time_dict = {
        "0": 0,
        "1": 0,
        "2": 0,
        "3": 20,
    }

    # weight of operation i in machine m
    # para_w = np.empty((n_opt, n_mach), dtype=object)
    para_w = np.full((n_mach, n_opt), dtype=object, fill_value=np.inf)

    # input/output delay time between two consecutive operations in mahcine m
    para_delta = np.empty((n_mach), dtype=object)

    # setup time of machine m when processing operation i before j
    # para_a = np.full((n_opt, n_opt, n_mach), dtype=object, fill_value=np.inf)
    para_a = np.full((n_mach, n_opt, n_opt), dtype=object, fill_value=np.inf)

    # capacity of machine
    para_mach_capacity = np.empty((n_mach), dtype=object)
    for m in range(n_mach):
        # capacity of machine is a set of constant numbers
        para_mach_capacity[m] = machine_data[str(m)]["c"]

        # input/output delay time between two consecutive operations in mahcine m
        # delta(m): loading and unloading time of machine m (=1 for all machines)
        para_delta[m] = 1

        # set up time of machine m when processing operation i before j
        # a(i,j,m): setup time of machine m when processing operation i before j (aijm = -inf if there
        # is no setups)
        for idx_setup, setup_data in enumerate(machine_data[str(m)]["setup_data"][0]):
            para_a[m, int(setup_data[0]), int(setup_data[1])] = setup_data[2]

        # maximum holding time of operation i in machine m
        para_h[:, m] = holding_time_dict[str(machine_data[str(m)]["t"])]

    # lag time
    for i in range(n_opt):
        for idx_lag, lag_data in enumerate(operation_data[str(i)]["lag"]):
            # minimum lag between the starting time of operation i and the ending time of operation j
            para_lmin[i, int(lag_data[0])] = lag_data[1]
            # maximum lag between the starting time of operation i and the ending time of operation j
            para_lmax[i, int(lag_data[0])] = lag_data[2]

        for idx_pw, pw_data in enumerate(operation_data[str(i)]["pw"]):
            # operation_data[str(1)]["pw"]
            # # the shape of para_p in the original file is the transpose of the shape of para_p
            # para_p[i, int(pw_data[0])] = pw_data[1]
            # # the shape of para_w in the original file is the transpose of the shape of para_w
            # para_w[i, int(pw_data[0])] = pw_data[2]

            # the shape of para_p in the original file is the transpose of the shape of para_p
            para_p[int(pw_data[0]), i] = pw_data[1]
            # the shape of para_w in the original file is the transpose of the shape of para_w
            para_w[int(pw_data[0]), i] = pw_data[2]

    # reformat the shape of para_p and para_w
    para_p = para_p.T
    para_w = para_w.T

    # # reshape the shape of para_a
    # para_a = np.einsum("mij->ijm", para_a)

    # the big M value
    big_m = get_m_value(
        para_p=para_p, para_h=para_h, para_lmin=para_lmin, para_a=para_a
    )

    return (
        n_opt,
        n_mach,
        operation_data,
        machine_data,
        para_lmin,
        para_lmax,
        para_p,
        para_h,
        para_w,
        para_delta,
        para_a,
        para_mach_capacity,
        big_m,
    )

In [4]:
n_opt_selected = 50
method = "CP"

In [5]:
    (
        n_opt,
        n_mach,
        operation_data,
        machine_data,
        para_lmin,
        para_lmax,
        para_p,
        para_h,
        para_w,
        para_delta,
        para_a,
        para_mach_capacity,
        big_m,
    ) = load_data()

    n_opt = n_opt_selected

    para_p_horizon = np.copy(para_p[:n_opt_selected, :])
    para_p_horizon[para_p_horizon == np.inf] = 0

    # para_h = np.empty((n_opt, n_mach), dtype=object)
    para_h_horizon = np.copy(para_h[:n_opt_selected, :])
    para_h_horizon[para_h_horizon == np.inf] = 0

    para_lmax_horizon = np.copy(para_lmax[:n_opt_selected, :n_opt_selected])
    para_lmax_horizon[para_lmax_horizon == np.inf] = 0

    horizon = (
        np.sum(para_p_horizon, axis=1)
        + np.sum(para_h_horizon, axis=1)
        + np.sum(para_lmax_horizon, axis=1)
    )
    horizon = int(np.sum(horizon)) + 1

    para_lmin[para_lmin == -np.inf] = -infinity
    para_lmin[para_lmin == np.inf] = infinity
    para_lmax[para_lmax == np.inf] = infinity
    para_lmax[para_lmax == -np.inf] = -infinity

    para_p[para_p == np.inf] = infinity
    para_p[para_p == -np.inf] = -infinity
    para_w[para_w == np.inf] = infinity
    para_w[para_w == -np.inf] = -infinity

    para_a[para_a == np.inf] = infinity
    para_a[para_a == -np.inf] = -infinity

    # =====================================================
    # convert all the numpy arrays with integers data type
    para_lmin = para_lmin.astype(int)
    para_lmax = para_lmax.astype(int)
    para_p = para_p.astype(int)
    para_w = para_w.astype(int)
    para_a = para_a.astype(int)
    para_delta = para_delta.astype(int)
    para_mach_capacity = para_mach_capacity.astype(int)

    para_lmin = para_lmin[:n_opt_selected, :n_opt_selected]
    para_lmax = para_lmax[:n_opt_selected, :n_opt_selected]

    para_p = para_p[:n_opt_selected, :]
    para_h = para_h[:n_opt_selected, :]
    para_w = para_w[:n_opt_selected, :]

    para_a = para_a[:n_opt_selected, :n_opt_selected, :]

    # machines
    machines = [str(i) for i in range(6)]
    # operations
    operations = [str(i) for i in range(n_opt_selected)]

    operations = operations[:n_opt_selected]
    machines = machines[:n_mach]

    # =====================================================
    if method == "CP":
        # time the running time for the CP model
        start_time = time.time()
        fjss2 = FJSS2(
            operations=operations,
            machines=machines,
            para_p=para_p,
            para_a=para_a,
            para_w=para_w,
            para_h=para_h,
            para_delta=para_delta,
            para_mach_capacity=para_mach_capacity,
            para_lmin=para_lmin,
            para_lmax=para_lmax,
            precedence=None,
            model_string=None,
            inf_cp=1.0e6,
            num_workers=16,
            verbose=True,
        )
        fjss2.build_model_ortools()
        fjss2_solution = fjss2.solve_ortools()
        # get the running time in seconds
        running_time = time.time() - start_time

        new_row = {
            "method": method,
            "n_opt": n_opt_selected,
            "n_mach": n_mach,
            "running_time_seconds": running_time,
            "num_constraints": 0,
            "makespan": fjss2.var_c_max,
        }
    elif method == "MILP":
        para_a = np.einsum("mij->ijm", para_a)
        # time the running time for the MILP model
        start_time = time.time()
        fjss3 = FJSS3(
            operations=operations,
            machines=machines,
            para_p=para_p,
            para_a=para_a,
            para_w=para_w,
            para_h=para_h,
            para_delta=para_delta,
            para_mach_capacity=para_mach_capacity,
            para_lmin=para_lmin,
            para_lmax=para_lmax,
            precedence=None,
            model_string=None,
            inf_milp=1.0e7,
            num_workers=16,
            verbose=True,
        )
        fjss3.solve_gurobi()
        running_time = time.time() - start_time
        new_row = {
            "method": method,
            "n_opt": n_opt_selected,
            "n_mach": n_mach,
            "running_time_seconds": running_time,
            "num_constraints": fjss3.solver.NumConstraints(),
            "makespan": fjss3.var_c_max,
        }
    else:
        raise ValueError("Invalid method!")


Starting CP-SAT solver v9.8.3296
Parameters: log_search_progress: true num_search_workers: 16

Initial optimization model '': (model_fingerprint: 0xcf42e113342f4bbe)
#Variables: 16'925'201 (#ints: 1 in objective)
  - 10'178'700 Booleans in [0,1]
  - 29'501 in [0,11195]
  - 6'717'000 in [0,1000000]
#kBoolAnd: 14'700 (#enforced: 14'700) (#literals: 44'100)
#kBoolOr: 44'100 (#enforced: 44'100) (#literals: 58'800)
#kIntProd: 3'358'500
#kLinMax: 29'400 (#expressions: 58'800)
#kLinear1: 20'151'000 (#enforced: 20'151'000)
#kLinear2: 6'839'550 (#enforced: 6'834'600) (#complex_domain: 14'700)
#kLinear3: 58'800 (#enforced: 58'800)
#kLinearN: 67'320 (#terms: 3'359'600)

Starting presolve at 9.09s
  1.66e+01s  0.00e+00d  [DetectDominanceRelations] 
  4.94e+02s  0.00e+00d  [PresolveToFixPoint] #num_loops=22 #num_dual_strengthening=5 
  7.86e-01s  0.00e+00d  [ExtractEncodingFromLinear] #potential_supersets=33'569 
[Symmetry] Problem too large. Skipping. You can use symmetry_level:3 or more to force

In [6]:
solver = fjss2._solver
model = fjss2._model

In [7]:
var_y = fjss2.var_y

# for y in solver.Value(var_y):
#     if  y== 1:
#         print(y)

[y for y in var_y.flatten() if solver.Value(y) == 1]

[y_0_0(0..1),
 y_1_4(0..1),
 y_2_1(0..1),
 y_3_4(0..1),
 y_4_2(0..1),
 y_5_4(0..1),
 y_6_2(0..1),
 y_7_4(0..1),
 y_8_2(0..1),
 y_9_0(0..1),
 y_10_5(0..1),
 y_11_1(0..1),
 y_12_5(0..1),
 y_13_2(0..1),
 y_14_5(0..1),
 y_15_2(0..1),
 y_16_5(0..1),
 y_17_2(0..1),
 y_18_5(0..1),
 y_19_2(0..1),
 y_20_0(0..1),
 y_21_4(0..1),
 y_22_1(0..1),
 y_23_4(0..1),
 y_24_2(0..1),
 y_25_4(0..1),
 y_26_2(0..1),
 y_27_4(0..1),
 y_28_2(0..1),
 y_29_0(0..1),
 y_30_3(0..1),
 y_31_1(0..1),
 y_32_3(0..1),
 y_33_2(0..1),
 y_34_0(0..1),
 y_35_3(0..1),
 y_36_1(0..1),
 y_37_3(0..1),
 y_38_2(0..1),
 y_39_3(0..1),
 y_40_2(0..1),
 y_41_3(0..1),
 y_42_2(0..1),
 y_43_3(0..1),
 y_44_2(0..1),
 y_45_0(0..1),
 y_46_3(0..1),
 y_47_1(0..1),
 y_48_3(0..1),
 y_49_2(0..1)]

In [8]:
# start time of operation i in machine m
var_s = fjss2.var_s

[solver.Value(s) for s in var_s]

[50,
 72,
 371,
 386,
 479,
 496,
 565,
 578,
 646,
 0,
 5,
 333,
 338,
 424,
 434,
 515,
 525,
 606,
 616,
 697,
 40,
 100,
 399,
 414,
 490,
 500,
 577,
 587,
 659,
 30,
 35,
 321,
 328,
 399,
 10,
 27,
 238,
 253,
 317,
 327,
 374,
 393,
 454,
 470,
 540,
 20,
 34,
 248,
 256,
 328]

In [9]:
# completion time of operation i in machine m
var_c = fjss2.var_c

[solver.Value(c) for c in var_s]

[50,
 72,
 371,
 386,
 479,
 496,
 565,
 578,
 646,
 0,
 5,
 333,
 338,
 424,
 434,
 515,
 525,
 606,
 616,
 697,
 40,
 100,
 399,
 414,
 490,
 500,
 577,
 587,
 659,
 30,
 35,
 321,
 328,
 399,
 10,
 27,
 238,
 253,
 317,
 327,
 374,
 393,
 454,
 470,
 540,
 20,
 34,
 248,
 256,
 328]

In [10]:
# var_u = fjss2.var_u

fjss2.var_u

array([[[u_0_0_0(0..1000000), u_0_0_1(0..1000000), u_0_0_2(0..1000000),
         ..., u_0_0_11192(0..1000000), u_0_0_11193(0..1000000),
         u_0_0_11194(0..1000000)],
        [u_0_1_0(0..1000000), u_0_1_1(0..1000000), u_0_1_2(0..1000000),
         ..., u_0_1_11192(0..1000000), u_0_1_11193(0..1000000),
         u_0_1_11194(0..1000000)],
        [u_0_2_0(0..1000000), u_0_2_1(0..1000000), u_0_2_2(0..1000000),
         ..., u_0_2_11192(0..1000000), u_0_2_11193(0..1000000),
         u_0_2_11194(0..1000000)],
        [u_0_3_0(0..1000000), u_0_3_1(0..1000000), u_0_3_2(0..1000000),
         ..., u_0_3_11192(0..1000000), u_0_3_11193(0..1000000),
         u_0_3_11194(0..1000000)],
        [u_0_4_0(0..1000000), u_0_4_1(0..1000000), u_0_4_2(0..1000000),
         ..., u_0_4_11192(0..1000000), u_0_4_11193(0..1000000),
         u_0_4_11194(0..1000000)],
        [u_0_5_0(0..1000000), u_0_5_1(0..1000000), u_0_5_2(0..1000000),
         ..., u_0_5_11192(0..1000000), u_0_5_11193(0..1000000),
         

## Checking the constraints


In [14]:
# # check eq. (22)

# for i, j, m in product(range(n_opt_selected), range(n_opt_selected), range(n_mach)):
#     if i != j:
#         if solver.Value(var_y[i, m]) == 1 and solver.Value(var_y[j, m]) == 1:
#             bool_left = (
#                 solver.Value(var_s[j]) >= solver.Value(var_c[i]) + para_a[m, i, j]
#             )
#             bool_right = (
#                 solver.Value(var_s[i]) >= solver.Value(var_c[j]) + para_a[m, j, i]
#             )
#             assert bool_left or bool_right

# # check eq. (23)

# for i, j, m in product(range(n_opt_selected), range(n_opt_selected), range(n_mach)):
#     if i != j:
#         if solver.Value(var_y[i, m]) == 1 and solver.Value(var_y[j, m]) == 1:
#             bool_left = (
#                 solver.Value(var_s[j])
#                 >= solver.Value(var_s[i])
#                 + max(
#                     [
#                         0,
#                         solver.Value(var_c[i])
#                         - solver.Value(var_s[i])
#                         - solver.Value(var_c[j])
#                         + solver.Value(var_s[j]),
#                     ]
#                 )
#                 + para_delta[m]
#             )
#             bool_right = (
#                 solver.Value(var_s[i])
#                 >= solver.Value(var_s[j])
#                 + max(
#                     [
#                         0,
#                         solver.Value(var_c[j])
#                         - solver.Value(var_s[j])
#                         - solver.Value(var_c[i])
#                         + solver.Value(var_s[i]),
#                     ]
#                 )
#                 + para_delta[m]
#             )
#             assert bool_left or bool_right

In [15]:
var_c_max

707.0

In [ ]:
model.ExportToFile("CP_model_50.pb.txt")

True

In [ ]:
# del check_constraints_milp
# del check_constraints_cp

In [16]:
from checking_constraints import check_constraints_milp, check_constraints_cp

# import importlib
# importlib.reload(check_constraints_milp)

%load_ext autoreload
%autoreload 2

In [17]:
def get_values(var):
    return solver.Value(var)


v_get_values = np.vectorize(get_values)

var_y = v_get_values(fjss2.var_y)
var_s = v_get_values(fjss2.var_s)
var_c = v_get_values(fjss2.var_c)
var_c_max = fjss2.var_c_max
var_u = v_get_values(fjss2.var_u)
num_t = int(fjss2.horizon / 1.0e0)

print(var_c_max)

# check if solutions from CP satisfy the constraints of CP formulation
check_constraints_cp(
    var_y=var_y,
    var_s=var_s,
    var_c=var_c,
    var_c_max=var_c_max,
    var_u=var_u,
    operations=operations,
    machines=machines,
    para_p=para_p,
    para_a=para_a,
    para_w=para_w,
    para_h=para_h,
    para_delta=para_delta,
    para_mach_capacity=para_mach_capacity,
    para_lmin=para_lmin,
    para_lmax=para_lmax,
    num_t=num_t,
)

707.0


In [18]:
# check if the solution from CP still satisfies the constraints of MILP
check_constraints_milp(
    var_y=var_y,
    var_s=var_s,
    var_c=var_c,
    var_c_max=var_c_max,
    operations=operations,
    machines=machines,
    para_p=para_p,
    para_a=np.einsum("mij->ijm", para_a),
    para_w=para_w,
    para_h=para_h,
    para_delta=para_delta,
    para_mach_capacity=para_mach_capacity,
    para_lmin=para_lmin,
    para_lmax=para_lmax,
    big_m=big_m,
    var_x=None,
    var_z=None,
)

In [ ]:
# %autoreload 0 - disables the auto-reloading. This is the default setting.
# %autoreload 1 - it will only auto-reload modules that were imported using the %aimport function (e.g %aimport my_module). It's a good option if you want to specifically auto-reload only a selected module.

# %autoreload 2 - auto-reload all the modules. Great way to make writing and testing your modules much easier.

In [26]:
from fjss import FjsOutput, SolvedOperation

assignments = dict()
start_times = dict()
end_times = dict()
solved_operations = []

# Task="Job A", Start='2009-01-01', Finish='2009-02-28', Completion_pct=50
df = pd.DataFrame(columns=["operation", "machine", "start_time", "end_time"])

for i, m in product(range(len(fjss2.operations)), range(len(fjss2.machines))):
    if solver.Value(fjss2.var_y[i, m]) == 1:
        assignments[fjss2.operations[i]] = fjss2.machines[m]
        start_times[fjss2.operations[i]] = solver.Value(fjss2.var_s[i])
        end_times[fjss2.operations[i]] = solver.Value(fjss2.var_c[i])
        solved_operation = SolvedOperation(
            id=fjss2.operations[i],
            assigned_to=fjss2.machines[m],
            start_time=solver.Value(fjss2.var_s[i]),
            end_time=solver.Value(fjss2.var_c[i]),
        )
        solved_operations.append(solved_operation)

        df.loc[i, "operation"] = fjss2.operations[i]
        df.loc[i, "machine"] = fjss2.machines[m]
        df.loc[i, "start_time"] = solver.Value(fjss2.var_s[i])
        df.loc[i, "end_time"] = solver.Value(fjss2.var_c[i])

fjs_output = FjsOutput(
    solved_operations=solved_operations,
    makespan=solver.ObjectiveValue(),
)

In [25]:
fjs_output

FjsOutput(solved_operations=[SolvedOperation(id='0', assigned_to='0', start_time=50.0, end_time=55.0), SolvedOperation(id='1', assigned_to='4', start_time=72.0, end_time=366.0), SolvedOperation(id='2', assigned_to='1', start_time=371.0, end_time=376.0), SolvedOperation(id='3', assigned_to='4', start_time=386.0, end_time=474.0), SolvedOperation(id='4', assigned_to='2', start_time=479.0, end_time=489.0), SolvedOperation(id='5', assigned_to='4', start_time=496.0, end_time=564.0), SolvedOperation(id='6', assigned_to='2', start_time=565.0, end_time=575.0), SolvedOperation(id='7', assigned_to='4', start_time=578.0, end_time=646.0), SolvedOperation(id='8', assigned_to='2', start_time=646.0, end_time=656.0), SolvedOperation(id='9', assigned_to='0', start_time=0.0, end_time=5.0), SolvedOperation(id='10', assigned_to='5', start_time=5.0, end_time=329.0), SolvedOperation(id='11', assigned_to='1', start_time=333.0, end_time=338.0), SolvedOperation(id='12', assigned_to='5', start_time=338.0, end_ti

In [27]:
df

,operation,machine,start_time,end_time
0,0,0,50,55
1,1,4,72,366
2,2,1,371,376
3,3,4,386,474
4,4,2,479,489
5,5,4,496,564
6,6,2,565,575
7,7,4,578,646
8,8,2,646,656
9,9,0,0,5


In [36]:
df.to_csv("CP_for_gantt_chart.csv")